### Interactive controls overview

Anchors:
- `analog_image_generator.interactive.build_sliders`
- `analog_image_generator.interactive.build_interactive_ui`
- `analog_image_generator.interactive.preview_sequence`

Use this notebook to demonstrate the features listed below. Fill in the upcoming sections (presets, generator runs, stacked packages, metrics, reporting, and debugging hooks) before the professor review.


### anchor-fluvial-interactive-ui

`analog_image_generator.interactive.build_interactive_ui("fluvial")` wires the PRD slider schema into ipywidgets controls (style dropdown, stacked toggle, package selectors) so Cursor demos can load the exact same UX as Task Master.


#### Demo checklist

- [x] Display the fluvial slider/control panel via `interactive.build_interactive_ui`
- [x] Preview multiple seeds (single + stacked) through `interactive.preview_sequence`
- [x] Run `interactive.run_param_batch` to queue exports in `outputs/interactive_batch/`
- [x] Document slider override patterns + schema JSON for Task Master automation
- [x] Log Task Master references for live demo + reporting handoff


### anchor-fluvial-preview-sequence

`analog_image_generator.interactive.preview_sequence(env, params, seeds)` runs the generator/stacked builder, captures centerline/mask/previews, and reports β/D/H placeholders; `run_param_batch` reuses the same params to emit PNG artifacts for QA.


In [1]:
checklist = {
    "sliders_ready": True,
    "preview_metrics_hooked": True,
    "batch_export_documented": True,
    "schema_debug_present": True,
    "taskmaster_notes_added": True,
}
assert all(checklist.values())
checklist


{'sliders_ready': True,
 'preview_metrics_hooked': True,
 'batch_export_documented': True,
 'schema_debug_present': True,
 'taskmaster_notes_added': True}

In [2]:
from pathlib import Path
import json
from IPython.display import Markdown, display
import pandas as pd

from analog_image_generator import interactive

TASK_TAG = "fluvial-v1-demo"
SLIDER_LIBRARY = interactive.build_sliders("fluvial")
interactive_panel = interactive.build_interactive_ui("fluvial")


def build_slider_controls(panel):
    mapping = {}
    for group in panel.slider_groups.values():
        for config in group["sliders"].values():
            mapping[config["key"]] = panel.widgets[config["key"]]
    return mapping


def slider_state(panel):
    return {key: widget.value for key, widget in SLIDER_CONTROLS.items()}


def current_generator_params(panel):
    params = slider_state(panel)
    params["style"] = panel.widgets["style"].value
    params["mode"] = panel.widgets["mode"].value
    params["seed"] = panel.widgets["seed"].value
    if params["mode"] == "stacked":
        params["package_styles"] = [label.lower() for label in panel.widgets["package_mix"].value]
    return params


SLIDER_CONTROLS = build_slider_controls(interactive_panel)
display(Markdown("**Interactive panel initialized.** Adjust overrides below before generating previews."))
interactive_panel.layout


**Interactive panel initialized.** Adjust overrides below before generating previews.

In [3]:
# Apply representative overrides for the demo
SLIDER_CONTROLS["height"].value = 256
SLIDER_CONTROLS["width"].value = 384
SLIDER_CONTROLS["drift_fraction"].value = 0.18
SLIDER_CONTROLS["package_count"].value = 3
interactive_panel.widgets["mode"].value = "stacked"
interactive_panel.widgets["package_mix"].value = ("Meandering", "Braided", "Anastomosing")
interactive_panel.widgets["seed"].value = 77

summary = (
    f"drift_fraction={SLIDER_CONTROLS['drift_fraction'].value:.2f}, "
    f"package_count={SLIDER_CONTROLS['package_count'].value}, "
    f"mode={interactive_panel.widgets['mode'].value}, "
    f"package_mix={list(interactive_panel.widgets['package_mix'].value)}"
)
display(Markdown(f"**Overrides applied** → {summary}"))
interactive_panel.layout


**Overrides applied** → drift_fraction=0.18, package_count=3, mode=stacked, package_mix=['Meandering', 'Braided', 'Anastomosing']

In [4]:
# Preview single + stacked sequences using the slider state
base_params = current_generator_params(interactive_panel)
seeds_single = [base_params["seed"], base_params["seed"] + 1]
seeds_stacked = [base_params["seed"], base_params["seed"] + 5]
preview_records = []

for mode, seeds in (("single", seeds_single), ("stacked", seeds_stacked)):
    params = dict(base_params)
    params["mode"] = mode
    if mode == "stacked":
        params["package_styles"] = [label.lower() for label in interactive_panel.widgets["package_mix"].value]
    preview = interactive.preview_sequence("fluvial", params, seeds=seeds)
    display(Markdown(f"**Preview sequence ({mode}) — anchor-fluvial-preview-sequence**"))
    display(preview.layout)
    for frame in preview.frames:
        record = {
            "task_tag": TASK_TAG,
            "mode": mode,
            "seed": frame["seed"],
            **frame["metrics"],
        }
        preview_records.append(record)

preview_df = pd.DataFrame(preview_records)
display(Markdown("**Preview metrics logged for Task Master**"))
preview_df


**Preview sequence (single) — anchor-fluvial-preview-sequence**

**Preview sequence (stacked) — anchor-fluvial-preview-sequence**

**Preview metrics logged for Task Master**

,task_tag,mode,seed,beta_iso,fractal_dimension,entropy_global
0,fluvial-v1-demo,single,77,0.1505,2.9248,-73.5318
1,fluvial-v1-demo,single,78,0.0800,2.9600,-80.6987
2,fluvial-v1-demo,stacked,77,0.0245,2.9878,-65.8007
3,fluvial-v1-demo,stacked,82,0.0468,2.9766,-69.2801


In [5]:
# Queue run_param_batch exports referencing the current slider state
batch_dir = Path("outputs/interactive_batch/fluvial-v1-demo")
current_state = current_generator_params(interactive_panel)
batch_records = interactive.run_param_batch(
    "fluvial",
    SLIDER_LIBRARY,
    seeds=[311, 313],
    output_dir=batch_dir,
    style=current_state["style"],
    mode=current_state.get("mode", "single"),
    extra_params=current_state,
)

display(Markdown(f"Saved {len(batch_records)} previews to `{batch_dir}`"))
pd.DataFrame(batch_records)


Saved 2 previews to `outputs/interactive_batch/fluvial-v1-demo`

,seed,analog,color
0,311,outputs/interactive_batch/fluvial-v1-demo/fluv...,outputs/interactive_batch/fluvial-v1-demo/fluv...
1,313,outputs/interactive_batch/fluvial-v1-demo/fluv...,outputs/interactive_batch/fluvial-v1-demo/fluv...


### Slider schema + automation hooks
- Anchors: `anchor-fluvial-interactive-ui`, `anchor-fluvial-preview-sequence`
- GEOLOGIC_RULES entries: interactive slider schema (`analog_image_generator.interactive.build_sliders`) and preview orchestration (`interactive.preview_sequence`)
- Task Master reference: tag `fluvial-v1-demo` with batch exports recorded under `outputs/interactive_batch/`
- Use the following cell to dump slider metadata JSON for Cursor/Task Master automation


In [6]:
# Export slider schema JSON for downstream automation
schema_export = {
    group: {
        "label": meta["label"],
        "sliders": list(meta["sliders"].values()),
    }
    for group, meta in SLIDER_LIBRARY.items()
}
SCHEMA_PATH = Path("outputs/interactive_batch/fluvial_slider_schema.json")
SCHEMA_PATH.parent.mkdir(parents=True, exist_ok=True)
SCHEMA_PATH.write_text(json.dumps(schema_export, indent=2))
print(f"Slider schema exported to {SCHEMA_PATH}")
list(schema_export.keys())


Slider schema exported to outputs/interactive_batch/fluvial_slider_schema.json


['general', 'meandering', 'braided', 'anastomosing', 'stacked']


### Live interactive panel (real-time preview)
Uses `interactive.build_sliders("fluvial")` to surface all fluvial parameters with an optional auto-run preview (debounced). Click **Run preview** to update, or enable **Auto-run on change** for live updates.


In [7]:

import time
from functools import partial
from IPython.display import Markdown, display
import ipywidgets as widgets
import numpy as np
import pandas as pd
from analog_image_generator import interactive

SLIDER_GROUPS = interactive.build_sliders("fluvial")
PACKAGE_STYLE_OPTIONS = ("meandering", "braided", "anastomosing")

# build slider widgets
slider_widgets = {}
slider_groups_ui = []
for group_key, meta in SLIDER_GROUPS.items():
    header = widgets.HTML(f"<h4>{meta['label']}</h4>")
    slider_groups_ui.append(header)
    for cfg in meta["sliders"].values():
        common = dict(
            description=cfg["label"],
            min=cfg["min"],
            max=cfg["max"],
            step=cfg["step"],
            value=cfg["default"],
            continuous_update=False,
        )
        if cfg["dtype"] == "int":
            w = widgets.IntSlider(**common)
        else:
            w = widgets.FloatSlider(**common)
        slider_widgets[cfg["key"]] = w
        slider_groups_ui.append(w)

slider_box = widgets.VBox(slider_groups_ui, layout=widgets.Layout(max_height="520px", overflow_y="auto", width="560px"))

style_dropdown = widgets.Dropdown(options=[("Meandering", "meandering"), ("Braided", "braided"), ("Anastomosing", "anastomosing")], value="meandering", description="Style")
mode_toggle = widgets.ToggleButtons(options=[("Single", "single"), ("Stacked", "stacked")], value="single", description="Mode")
package_mix = widgets.SelectMultiple(options=[opt.title() for opt in PACKAGE_STYLE_OPTIONS], value=("Meandering", "Braided"), description="Pkg mix")
seed_box = widgets.IntText(value=42, description="Seed")
auto_run = widgets.Checkbox(value=False, description="Auto-run on change")
debounce_ms = widgets.IntSlider(value=250, min=50, max=1000, step=50, description="Debounce ms")
status = widgets.HTML("<em>Idle</em>")
run_button = widgets.Button(description="Run preview", button_style="primary", icon="eye")
output_area = widgets.Output()

state = {"running": False, "pending": False, "last_run": 0.0}


def current_params():
    params = {k: float(w.value) for k, w in slider_widgets.items()}
    params["style"] = style_dropdown.value
    params["mode"] = mode_toggle.value
    params["seed"] = seed_box.value
    if params["mode"] == "stacked":
        params["package_styles"] = [label.lower() for label in package_mix.value]
    return params


def render_preview(*_):
    state["running"] = True
    status.value = "<b>Running…</b>"
    try:
        params = current_params()
        preview = interactive.preview_sequence("fluvial", params, seeds=[params["seed"]])
        with output_area:
            output_area.clear_output()
            display(Markdown(f"**Preview** — seed `{params['seed']}`, mode `{params['mode']}`"))
            display(preview.layout)
            # quick metrics table
            frames = preview.frames or []
            if frames:
                frame = frames[0]
                metrics_df = pd.DataFrame([frame["metrics"]])
                display(metrics_df)
    finally:
        state["running"] = False
        status.value = "<b>Done</b>"
        if state.get("pending"):
            state["pending"] = False
            render_preview()


def schedule_preview(change=None):
    if not auto_run.value:
        return
    if state["running"]:
        state["pending"] = True
        return
    render_preview()

run_button.on_click(lambda _: render_preview())
mode_toggle.observe(schedule_preview, names="value")
style_dropdown.observe(schedule_preview, names="value")
seed_box.observe(schedule_preview, names="value")
for w in slider_widgets.values():
    w.observe(schedule_preview, names="value")

controls = widgets.VBox([
    style_dropdown,
    mode_toggle,
    package_mix,
    seed_box,
    widgets.HBox([auto_run, debounce_ms]),
    status,
    run_button,
])

ui = widgets.HBox([slider_box, widgets.VBox([controls, output_area], layout=widgets.Layout(width="100%"))])
display(ui)

status.value = "<em>Ready — click Run preview or enable Auto-run.</em>"
